# Student Loan Risk with Deep Learning

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Define the features set X using all columns except the target
X = loans_df.drop(columns=['credit_ranking'])

# Display a sample of y
print(y.head())


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


In [7]:
# Define features set X by selecting all columns but credit_ranking
# Done in previous step

# Review the features DataFrame
print(X.head())


   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Check the shape of the resulting datasets to confirm
print(f"Training data: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
print(f"Testing data: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")


Training data: X_train shape = (1279, 11), y_train shape = (1279,)
Testing data: X_test shape = (320, 11), y_test shape = (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Scale the testing dataset
X_test_scaled = scaler.transform(X_test)

# Review the scaled features for X_train_scaled
print(X_train_scaled[:5])


[[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]
 [-0.62052788  0.49684658 -1.05484864 -0.0345135  -0.20139592  0.11885252
   1.18937522  0.37546769  1.24352773 -0.76640023 -0.6873351 ]
 [-0.50797663  0.60887739 -1.00429969 -0.53491726  0.26101766 -0.74240736
  -0.7077067  -0.33312578 -0.09557544 -0.47015208 -0.77978452]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the number of inputs (features) to the model
number_of_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {number_of_features}")


Number of input features: 11


In [12]:
# Create the Sequential model instance
# Define the model
model = Sequential()

# Add the first hidden layer
# Define the number of hidden nodes for the first hidden layer
# First hidden layer (example: 16 neurons)
model.add(Dense(units=16, activation='relu', input_dim=number_of_features))

# Add the second hidden layer
# Define the number of hidden nodes for the second hidden layer
# Second hidden layer (example: 8 neurons)
model.add(Dense(units=8, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
# Define the number of neurons in the output layer
# Output layer (binary classification, so we use sigmoid activation)
model.add(Dense(units=1, activation='sigmoid'))



In [13]:
# Display the Sequential model summary
# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 16)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 337 (1.32 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.4816 - loss: 0.7126 - val_accuracy: 0.6719 - val_loss: 0.6576
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6458 - loss: 0.6577 - val_accuracy: 0.7063 - val_loss: 0.6206
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6959 - loss: 0.6157 - val_accuracy: 0.7063 - val_loss: 0.5894
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7082 - loss: 0.5869 - val_accuracy: 0.7188 - val_loss: 0.5610
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6999 - loss: 0.5749 - val_accuracy: 0.7188 - val_loss: 0.5432
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6882 - loss: 0.5680 - val_accuracy: 0.7312 - val_loss: 0.5305
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7262 - loss: 0.5308 - val_accuracy: 0.7281 - val_loss: 0.5224
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7535 - loss: 0.5161 - val_accuracy: 0.7375 - val_loss

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7758 - loss: 0.4613 
Test Loss: 0.507868766784668
Test Accuracy: 0.746874988079071


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [18]:
# Set the model's file path
model.save('student_loans.keras')

# Export your model to a keras file

# Verify that the model file is saved
!ls -lh student_loans.keras




-rw-r--r-- 1 root root 25K Sep 26 01:56 student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [20]:
from tensorflow.keras.models import load_model

# Set the model's file path
file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(file_path)

# Confirm the model has been loaded by printing its summary
loaded_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 16)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,013 (3.96 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 676 (2.64 KB)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [21]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Since we are using a sigmoid activation function, convert the predictions to binary (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame({
    'Predicted': binary_predictions.flatten(),
    'Actual': y_test
})

# Display a sample of the predictions
print(predictions_df.head())


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
      Predicted  Actual
75            0       0
1283          0       1
408           1       1
1281          1       1
1118          1       1


In [23]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Done in previous step


### Step 4: Display a classification report with the y test data and predictions

In [25]:
# Print the classification report with the y test data and predictions

# Generate the classification report using the true labels (y_test) and the binary predictions
report = classification_report(y_test, binary_predictions)

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       0.73      0.76      0.74       154
           1       0.77      0.73      0.75       166

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:


---



1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

---

***Financial History:*** Includes income level, existing debt, and credit score.

***Academic Information:*** The field of study, academic performance, and graduation likelihood.

***Loan Specifics:*** Amount needed, repayment terms desired, and interest rate sensitivity.

***Personal Factors:*** Employment status, family contributions, and future income potential.

---

***Relevance and Appropriateness:***


***Financial History:*** Essential for assessing a student’s ability to repay the loan.

***Academic Information:*** Provides insights into the student’s potential earning power, which is closely tied to their field and performance.

***Loan Specifics:*** Tailors recommendations to fit personal financial needs and capacities.

***Personal Factors:*** Offers a holistic view of the student’s financial ecosystem, crucial for customizing loan options.


---



2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.



---



***Content-based filtering*** would be the most suitable because this approach can utilize detailed attributes of both the students and the loan products to generate recommendations. This method matches loan types with personal and financial data of students, making the recommendations highly personalized.

***Justification:***

Content-based systems focus on the attributes of the items (in this case, loan terms and conditions) and a profile of the user’s preferences. This method ensures that the system can recommend loan options that are best aligned with the student’s specific financial and academic status.



---



3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.


---



***Data Privacy and Security:***

Concern: Handling sensitive financial and personal information requires stringent security measures to protect against data breaches.
Impact: Ensures trust in the system’s use by students who may be wary of providing personal information online.

***Bias and Fairness:***
Concern: Algorithms might inadvertently favor or disfavor certain demographics based on historical data, leading to biased lending practices.

Impact: Maintaining fairness is critical to avoid systemic biases that could affect loan accessibility for underrepresented or disadvantaged groups.

Advantages:
Provides tailored recommendations, potentially increasing satisfaction and fit for the user’s needs.
Can automate and streamline the process of finding suitable loan options, saving time and effort.

Disadvantages:
Implementing sophisticated data protection measures can be costly and complex.
Monitoring and updating algorithms to avoid biases can require ongoing oversight and adjustments, adding to operational complexities.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

